In [102]:
##### utils.py

import requests
import json
import time
import pandas as pd
import random
from datetime import datetime
from datetime import timedelta
import os
from lxml.html import fromstring
import re


### 지정된 검색어(1개)에 대해 해당 일자(From, To)의 기사 리스트를 가져옴.
def get_article_df(search_keyword, search_prd_from=None, search_prd_to=None, sort=0):
    ###검색 일자 (From, To)
    #검색 일자(from)가 None인 경우:
    if search_prd_from is None:
        #현재 일자를 가져옴 (형태: YYYY.MM.DD) - UTC와 KST 고려하면 +9시간 추가해야 함 (time_def=9)
        time_def = 9
        search_prd_from = (datetime.now() + timedelta(hours=time_def)).strftime("%Y.%m.%d")
    else:
        #입력된 값을 전처리함 (To-Be 형태: YYYY.MM.DD)
        search_prd_from = re.sub(r"[^0-9]", "", search_prd_from)
        search_prd_from = datetime.strptime(search_prd_from, "%Y%m%d").strftime("%Y.%m.%d")
               
    #검색 일자(to)가 None인 경우:
    if search_prd_to is None:
        #search_prd_to와 동일한 값 부여
        search_prd_to = search_prd_from
    else:
        #입력된 값을 전처리함 (To-Be 형태: YYYY.MM.DD)
        search_prd_to = re.sub(r"[^0-9]", "", search_prd_to)
        search_prd_to = datetime.strptime(search_prd_to, "%Y%m%d").strftime("%Y.%m.%d")
        #검색 일자(to)가 검색 일자(from)보다 이전인 경우:
        if search_prd_to < search_prd_from:
            #search_prd_to와 동일한 값 부여
            search_prd_to = search_prd_from
    
    
    ### URL 정의
    URL = "https://m.search.naver.com/search.naver"
    param = {
        'where': 'm_news',
        'sm': 'mtb_pge',
        'query': search_keyword,
        'sort': str(sort), #관련도 순 정렬: 0, 최신 순으로 정렬: 1
        'photo': '0',
        'field': '0',
        'pd': '3',
        'ds': search_prd_from,
        'de': search_prd_to,
        'mynews': '0',
        'office_type': '0',
        'office_section_code': '0',
    }#--param
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36',
        'Referer': 'https://m.news.naver.com/'
    }#--header

    ### 기사 검색 결과 크롤링 수행
    #while문으로 계속 반복해서 돌기 - 더 이상 다음 페이지번호가 나오지 않을 때까지
    page = 0
    resultset = pd.DataFrame()

    while True:
        page += 1
        this_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S') #서버 시간 기준
        
        if( page==1 ):
            resp = requests.get(URL, params=param, headers=header)
        else:
            #해당 페이지의 링크 확인
            page_range = doc.xpath('//div[@class="sp_page"]/div[@class="api_subject_bx"]/div[@class="page_wrap"]/div[@class="list_page"]/a/text()')
            if( str(page) in page_range ):
                this_idx = page_range.index(str(page))
                this_url = doc.xpath('//div[@class="sp_page"]/div[@class="api_subject_bx"]/div[@class="page_wrap"]/div[@class="list_page"]/a')[this_idx].get('href')
                this_url = URL + this_url
                resp = requests.get(this_url, headers=header)
            else:
                break

        #기사 리스트
        print(search_keyword, " - Page ", str(page))
        resp.encoding = 'utf-8'
        doc = fromstring(resp.text)
        news_range = doc.xpath('//*[@id="news_result_list"]/li[@class="bx"]')
        this_result = get_article_list(news_range)

        this_resultset = pd.DataFrame(data=this_result, columns=['i', 'press_name', 'article_time', 'article_naver_flag', 'article_title', 'article_link', 'article_abstract', 'article_rel_count'])
        this_resultset['search_keyword'] = search_keyword
        this_resultset['search_prd_from'] = search_prd_from
        this_resultset['search_prd_end'] = search_prd_to
        this_resultset['search_page_no'] = page
        this_resultset['search_timestamp'] = this_timestamp

        resultset = pd.concat([resultset, this_resultset], ignore_index=True)
        time.sleep(random.randrange(100, 200)*0.01) #기사 페이지마다 1~2초 사이로 pause

    return(resultset)
#-- def get_article_df()
    
### get_article_df 함수 내에서 사용되는 함수 (기사 리스트로부터 원하는 정보를 parsing)
def get_article_list(news_range):
    result_list = []
    for i in range(0, len(news_range)):
        this_news = news_range[i]
        this_press_name = this_news.xpath('.//div[@class="news_wrap"]/div[@class="news_info"]/div[@class="info_group"]/a/text()')[0]
        this_article_time = this_news.xpath('.//div[@class="news_wrap"]/div[@class="news_info"]/div[@class="info_group"]/span[1]/text()')[0]
        #this_article_naver_flag = this_news.xpath('.//div[@class="news_wrap"]/div[@class="news_info"]/div[@class="info_group"]/span[2]/text()')[0]
        
        this_article_naver_flag = this_news.xpath('.//div[@class="news_wrap"]/div[@class="news_info"]/div[@class="info_group"]/span[2]/text()')
        if len(this_article_naver_flag)>0: this_article_naver_flag = this_article_naver_flag[0]
        else: this_article_naver_flag = ''
        
        #this_article_title = ''.join((this_news.xpath('.//div[@class="news_wrap"]/a[@class="news_tit"]/div/text()')))
        this_article_title = this_news.xpath('.//div[@class="news_wrap"]/a[@class="news_tit"]/div')[0].text_content()
        this_article_link = this_news.xpath('.//div[@class="news_wrap"]/a[@class="news_tit"]')[0].get('href')
        this_article_abstract = this_news.xpath('.//div[@class="news_wrap"]/div[@class="news_dsc"]/div[@class="dsc_wrap"]/a/div[contains(@class, "dsc_txt")]')[0].text_content()
                
        #this_article_rel_count = this_news.xpath('.//a[@class="news_more"]/text()')[0].split(' ')[1].replace('건', '')
        this_article_rel_count = this_news.xpath('.//a[@class="news_more"]/text()')
        if len(this_article_rel_count)>0: this_article_rel_count = this_article_rel_count[0].split(' ')[1].replace('건', '')
        else: this_article_rel_count = ''        
        #this_article_rel_link = this_news.xpath('.//a[@class="news_more"]')[0].get('href')
        
        this_result_list = [i, this_press_name, this_article_time, this_article_naver_flag, this_article_title, this_article_link, this_article_abstract, this_article_rel_count]
        result_list.append(this_result_list)
    return(result_list)
#--def get_article_list()


### get_reply_list
def get_reply_list(url):
    this_article_url = url
    this_oid = this_article_url.split("oid=")[1].split("&aid")[0]
    this_aid = this_article_url.split("aid=")[1]
    this_page = 0

    resultset = pd.DataFrame()
    #resultset_list = list()

    while True:
        this_page += 1
        this_article_reply_url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=view_economy&pool=cbox5&_wr&_callback=jQuery33101823973869306712_1626419151371&lang=ko&country=KR&objectId=news" + this_oid + "%2C" + this_aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(this_page) + "&sort=favorite"
        header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36',
            'Referer': this_article_url
        }#--header
        this_response_reply = requests.get(this_article_reply_url, headers=header)
        this_response_reply.encoding = "UTF-8"

        #JSON Parsing
        this_doc_reply = this_response_reply.text
        this_idx_strt = this_doc_reply.find("(")
        this_idx_end = this_doc_reply.find(");")
        this_doc_reply = this_doc_reply[this_idx_strt+1:this_idx_end]
        this_doc_reply = json.loads(this_doc_reply)

        reply_range = this_doc_reply["result"]["commentList"] #댓글 수

        if len(reply_range)>0:    
            this_resultset = pd.DataFrame()
            for i in range(0, len(reply_range)):
                this_reply = reply_range[i]
                this_resultset = pd.concat([this_resultset, pd.DataFrame([this_reply])], ignore_index=True)
                #this_doc_reply["result"]["commentList"][0]["contents"] #0번째 댓글 내용
            #resultset_list.append(this_resultset)
            resultset = pd.concat([resultset, this_resultset], ignore_index=True)
        else:
            break
        
    return(resultset)
#--def get_reply_list()

In [66]:
### 특정 키워드에 대해 특정 기간 동안의 검색 결과를 가져오기
df = get_article_df(search_keyword="라이나생명", search_prd_from="20210719", search_prd_to="20210720", sort=0) #sort 0: 관련도 순 정렬, 1: 최신 순 정렬
df

# press_name: 언론사
# article_time: 게시일시 (crawling_timestamp 기준)
# article_naver_flag: 네이버뉴스 여부
# article_title: 기사명
# article_link: 기사 URL
# article_abstract: 기사 요약 (네이버 제공)
# article_rel_count: 관련기사 수 (네이버 제공)
# search_keyword: 크롤링 검색 키워드 (함수 파라미터)
# search_prd_from: 크롤링 검색 시작일 (함수 파라미터)
# search_prd_to: 크롤링 검색 종료일 (함수 파라미터)
# search_page_no: 크롤링 결과 페이지 번호
# search_timestamp: 크롤링 일시 (서버시간 기준)

라이나생명  - Page  1


,i,press_name,article_time,article_naver_flag,article_title,article_link,article_abstract,article_rel_count,search_keyword,search_prd_from,search_prd_end,search_page_no,search_timestamp
0,0,이데일리,1일 전,네이버뉴스,"라이나생명, 사내어린이집 평가 인증 `최고 등급` 획득",https://m.news.naver.com/read.nhn?mode=LSD&mid...,라이나생명보험 사내 어린이집 ‘키즈나루’ 모습. 라이나생명 제공.[이데일리TV 이혜...,28,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
1,1,비즈월드,2시간 전,,"[보험 Life] 삼성생명, 푸르덴셜생명, NH농협생명, 라이나생명",http://www.bizwnews.com/news/articleView.html?...,"사진=라이나생명 ◆라이나생명, 사내 어린이집 평가 인증 'A등급' 획득 라이나생명보...",,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
2,2,아시아타임즈,18시간 전,,"[보험 현미경] AXA손보, '차량 무상점검 서비스' 실시",https://m.asiatime.co.kr/article/20210719500342,"■ 라이나생명 사내어린이집, '최고 등급' 획득 라이나생명의 사내어린이집이 한국보육...",,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
3,3,헤럴드경제,1시간 전,네이버뉴스,"아크릴, AI 의료서비스 공동 개발 착수",https://m.news.naver.com/read.nhn?mode=LSD&mid...,"현재 삼성서울병원, 가톨릭대 서울성모병원, 연세대 세브란스병원, 서울대병원 등의 의...",,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
4,4,이코노미스트,1시간 전,네이버뉴스,"[단독] '코로나 백신보험' 봇물…생보협 ""명칭 제대로 써라""",https://m.news.naver.com/read.nhn?mode=LSD&mid...,'코로나 백신보험 이벤트' 봇물…우려 커지자 조치 보험업계에 따르면 생보협회는 지난...,,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
5,5,뉴시스,23분 전,네이버뉴스,"아크릴-한림대병원-파인헬스케어, AI 의료 서비스 협력",https://m.news.naver.com/read.nhn?mode=LSD&mid...,"특히 바이오헬스 및 보험산업 혁신 기술인 인슈어테크 영역에서 삼성서울병원, 가톨릭대...",,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
6,6,보험매일,6시간 전,,상반되는 생·손보 전속설계사 수...GA 는 '증가세',http://www.fins.co.kr/news/articleView.html?id...,"한화생명 외 삼성생명, 신한라이프, 미래에셋생명, 라이나생명, 메트라이프생명, AB...",,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
7,7,EBN,3시간 전,,"4세대 실손 점입가경…보험사, 손해율 독박?",https://www.ebn.co.kr/news/view/1492577/?sc=Naver,라이나생명이 2011년부터 실손보험 판매를 중단했다. 이후 △오렌지라이프(2012년...,,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
8,8,전자신문,1시간 전,네이버뉴스,"아크릴, 한림대강남성심병원·파인헬스케어와 AI 의료 서비스 공동 개발",https://m.news.naver.com/read.nhn?mode=LSD&mid...,"삼성서울병원, 가톨릭대 서울성모병원, 연세대 세브란스병원, 서울대병원 등 의료기관을...",,라이나생명,2021.07.19,2021.07.20,1,20210720_154651
9,9,이데일리,23시간 전,네이버뉴스,"""실손보험 가입문턱 높이지마”…금융당국, 보험사에 경고",https://m.news.naver.com/read.nhn?mode=LSD&mid...,"AIA생명과 오렌지라이프, 라이나생명 등은 2011~2013년에 걸쳐 일찌감치 실손...",,라이나생명,2021.07.19,2021.07.20,1,20210720_154651


In [103]:
### 댓글 가져오기 (네이버뉴스만 가능)
this_url = df.article_link[9]
df_url = get_reply_list(this_url)
df_url

,ticket,objectId,categoryId,templateId,commentNo,parentCommentNo,replyLevel,replyCount,replyAllCount,replyPreviewNo,...,idNo,blindReport,blind,serviceId,containText,userBlocked,maskedUserId,maskedUserName,validateBanWords,anonymous
0,news,"news018,0004989045",*,view_economy,2624033964,2624033964,1,6,6,None,...,3Okf9,False,False,None,True,False,cred****,cr****,False,False
1,news,"news018,0004989045",*,view_economy,2624050794,2624050794,1,0,0,None,...,i7QJ,False,False,None,True,False,shin****,sh****,False,False
2,news,"news018,0004989045",*,view_economy,2624809334,2624809334,1,0,0,None,...,6lFto,False,False,None,True,False,ysoo****,ys****,False,False
3,news,"news018,0004989045",*,view_economy,2624088304,2624088304,1,1,1,None,...,5DuSJ,False,False,None,True,False,tjch****,tj****,False,False
4,news,"news018,0004989045",*,view_economy,2624034654,2624034654,1,0,0,None,...,5mKV4,False,False,None,True,False,seba****,se****,False,False


In [104]:
#댓글 수
len(df_url)

5

In [113]:
df_url.columns

Index(['ticket', 'objectId', 'categoryId', 'templateId', 'commentNo',
       'parentCommentNo', 'replyLevel', 'replyCount', 'replyAllCount',
       'replyPreviewNo', 'replyList', 'imageCount', 'imageList',
       'imagePathList', 'imageWidthList', 'imageHeightList', 'commentType',
       'stickerId', 'sticker', 'sortValue', 'contents', 'userIdNo',
       'exposedUserIp', 'lang', 'country', 'idType', 'idProvider', 'userName',
       'userProfileImage', 'profileType', 'modTime', 'modTimeGmt', 'regTime',
       'regTimeGmt', 'sympathyCount', 'antipathyCount', 'hideReplyButton',
       'status', 'mine', 'best', 'mentions', 'toUser', 'userStatus',
       'categoryImage', 'open', 'levelCode', 'grades', 'sympathy', 'antipathy',
       'snsList', 'metaInfo', 'extension', 'audioInfoList', 'translation',
       'report', 'middleBlindReport', 'spamInfo', 'userHomepageUrl',
       'defamation', 'hiddenByCleanbot', 'score', 'ratings', 'managerLike',
       'visible', 'manager', 'deleted', 'expose',

In [116]:
#댓글 내용
df_url.contents

0       보험사 이것들은 매년 성과급 잔치하면서 고객과 거래는 1도 손해 안볼라고ᆢ더러운것들
1        보험갖고 장난치는 인간들 때문에 대다수의   선량한  사람들이  피해를 입는다 ㅠ
2        외래 안본 사람이면 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 작정하고 안전하게 돈뜯어먹겠다는 의지...
3    살손보험상품자체를없애라\n 얄팍한병원배만채우고있다\n 차리리국민건강보험을현실화해라\...
4                공산당이냐? 어찌 중국  따라가냐? 각 기업이 알아서 하는거야!!!
Name: contents, dtype: object